# ETL Pipeline For Disaster Response Message Classification

## Extract

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Load messages dataset
messages = pd.read_csv("messages.csv")
messages.head()

In [ ]:
# Load categories dataset
categories = pd.read_csv("categories.csv")
categories.head()

In [ ]:
# Merge datasets
df = messages.merge(categories, how="left", on="id")
df.head()

## Transform

In [ ]:
# Create a dataframe of the 36 individual category columns
categories = df.categories.str.split(";", expand=True)
categories.head()

In [ ]:
# Select the first row of the categories dataframe and use text as column names
row = categories.iloc[0].str.replace(r'[^a-z_]','').tolist()
category_colnames = row
print(category_colnames)

In [ ]:
# Rename the columns of categories
categories.columns = category_colnames
categories.head()

In [ ]:
for column in categories:
    # Set each value to be the last character of the string
    categories[column] = categories[column].astype(str).str.replace(r'[^0-9]','')
    
    # Convert column from string to numeric
    categories[column] = categories[column].astype(str).astype(int)
    
categories.head()

In [ ]:
# Drop the original categories column from `df`
df.drop("categories", axis=1, inplace=True)

df.head()

In [ ]:
# Concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

In [ ]:
# Check for weird looking categories such as
# other than binary values or zero variance
for column in df[category_colnames]:
    if df[column].unique().sum() != 1:
        print("Column name: {}, Unique values: {}".format(column, df[column].unique()))

In [ ]:
# Drop rows without binary categories
df = df[df["related"] != 2]

# Drop categories with zero variance
df.drop("child_alone", axis=1, inplace=True)
df.shape

In [ ]:
# Check number of duplicates
df.duplicated().sum()

In [ ]:
# Drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# Check number of duplicates
df.duplicated().sum()

## Load

In [ ]:
# Save df to SQLite databse
engine = create_engine('sqlite:///disaster_response.db')
df.to_sql('messages', engine, index=False)